找到每个cluster里面分数最低的每对向量组,qvecs对应的是qids提取的向量，vecs对应的是pidx提取的向量

In [18]:
import os
import torch
import numpy as np
import copy
import tqdm
import pickle
vecs=np.load('/root/server/EXPORT_DIR=petModel/pet_resnet101_gem_softmax_m0.70/_adam_lr1.0e-06_wd1.0e-04_gid1/'+
             '_nnum5_qsize17645_psize2000/_bsize16_imsize1024/model_epoch3/pet_vecs_ep3_resize.npy')
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
with open('/root/server/images/pet_train.pkl', 'rb') as f:
    gnd_train = pickle.load(f)
cids=gnd_train['train']['cids']
clu_dic=gnd_train['train']['clu_dic']
cluster=gnd_train['train']['cluster']
print(vecs.shape)

(2048, 5389)


In [10]:
##clu_vec里面应该存的是一个方阵
# clu_vec=copy.deepcopy(clu_dic)
new_qidx=[]
new_pidx=[]
num_clu_mou=0
all_cont=0
with torch.no_grad():
    for mou_clu in clu_dic:
        num_clu_mou+=1
        list_id=clu_dic[mou_clu]
        print (num_clu_mou,len(list_id))
        vecc=np.zeros([2048,len(list_id)],dtype=np.float32)
        vecc=np.zeros([2048,len(list_id)],dtype=np.float32)
        for j in range(len(list_id)):
            vecc[:,j]=vecs[:,list_id[j]]
        vecc = torch.from_numpy(vecc).cuda()
        vecc.requires_grad_(False)
        score=torch.mm(vecc.transpose(1,0),vecc)
        assert score.shape[0]==len(list_id) and score.shape[1]==len(list_id)
#             把现有的score矩阵拉伸为向量
        new_score=torch.zeros(int(score.shape[0]*(score.shape[0]-1)/2)).cuda()            
        new_iddui=[]                      
        count=0
        for i in range(0,score.shape[0]):
            for j in range(i+1,score.shape[1]):
                new_iddui.append([list_id[i],list_id[j]])
                new_score[count]=score[i][j] 
                count+=1
        assert new_score.shape[0]==len(new_iddui)
        ranks=torch.sort(new_score)[1]
        new_new_iddui=[]
        if len(ranks)>=370:
            for i in range(370):
                new_new_iddui.append(new_iddui[ranks[i]])
        else:
            for i in range(len(ranks)):
                new_new_iddui.append(new_iddui[ranks[i]])
        for i in new_new_iddui:
            new_qidx.append(i[0])
            new_pidx.append(i[1])
        torch.cuda.empty_cache()

1 199
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 199
14 200
15 200
16 200
17 200
18 191
19 200
20 200
21 200
22 200
23 200
24 200
25 200
26 200
27 200


In [19]:
for i in range(len(new_pidx)):
    assert cluster[new_pidx[i]]==cluster[new_qidx[i]]

In [20]:
gnd_train['train']['qidxs']=new_qidx
gnd_train['train']['pidxs']=new_pidx
with open("../images/pet_train_hard.pkl",'wb') as f:
    f.write( pickle.dumps(gnd_train))